# Obliczanie wartości sinusa w dziedzinie liczb zespolonych

### Podstawowe funkcje i parametry

In [24]:
import Base.+, Base.*, Base.-, Base./

using Plots

plotly()

Prec = BigFloat
Comp = Tuple{Prec, Prec}

SIN_MAX_ITER = 10
COS_MAX_ITER = 10
SINH_MAX_ITER = 50
COSH_MAX_ITER = 50

PRECISION = 128

setprecision(PRECISION)

# Podstawowe operacje na liczbach zespolonych
function +(x::Comp, y::Comp)
    return x[1] + y[1], x[2] + y[2]
end

function *(x::Comp, y::Comp)
    return  x[1] * y[1] - x[2] * y[2], x[1] * y[2] + x[2] * y[1]
end

function -(x::Comp)
    return -x[1], -x[2]
end

function /(x::Comp, n::Int64)
    return x[1] / n, x[2] / n
end

# Kwadrat liczby zespolonej
function square(x::Comp)
    return x * x
end

# Funkcja obliczająca n-tą potęgę liczby zespolonej,
# korzystając z algorytmu szybkiego potęgowania.
# Ogranicza do minimum wykonywaną liczbę mnożeń (w tym błąd).
function power_complex(x::Comp, n::Int64)
    if n > 0
        if n & 1 == 1
            return square(power_complex(x, n>>=1)) * x
        else
            return square(power_complex(x, n>>=1))
        end
    else
        return Comp((1, 0))
    end
end

# Funkcja obliczająca wartość symbolu newtona
function newton(n::Int64, k::Int64)
    if n < k return 0 end
    if k < 0 return 0 end
    Σ = Prec(1)
    for i = 1:(n-k)
         Σ = Σ * (i + k) / i
    end
    return Σ
end

function test(f::Function, z::Comp, max_iter::Int64)
    setprecision(1024)
    result = sin(z[1] + z[2] * im)
    res = BigFloat(real(result), 128), BigFloat(imag(result), 128)
    setprecision(128)
    
    @printf "%.10e " abs((prevfloat(res[1]) - res[1])/res[1])
    @printf "%.10e\n" abs((prevfloat(res[2]) - res[2])/res[2])
    
    MAX_ITER = max_iter
    f(z, res)
end

test (generic function with 1 method)

## Sposób 1
Podstawowym pomysłem jest obliczanie sinusa z szeregu Taylora, kolejno obliczając potęgi i sumując liczby zespolone. Podejście jest bardzo naiwne, ale dla liczb o małym module okazuje się dość dobre.

In [30]:
MAX_ITER = 30

function sin_complex_1(z::Comp, res)
    Σ  = z
    _z = z
    Δ_real = Array{BigFloat}(MAX_ITER)
    Δ_imag = Array{BigFloat}(MAX_ITER)
    for i = 1:MAX_ITER
        _z = - _z * z / (2 * i) * z / (2 * i + 1)
        Σ += _z
        Δ_real[i] = abs(Σ[1] - res[1])
        Δ_imag[i] = abs(Σ[2] - res[2])
        @printf "%.10e %.10e \n" abs((Σ[1] - res[1])/res[1]) abs((Σ[2] - res[2])/res[1])
    end
    return Σ, Δ_real, Δ_imag
end


sin_complex_1 (generic function with 2 methods)

### Przykład 1.1

In [33]:
res = test(sin_complex_1, Comp((2, 3)), 20)
plot([abs.(res[2][2:end] ./ res[2][1:end-1]))[3:end], abs.(res[3][2:end] ./ res[3][1:end-1])[3:end]], title="Iloraz błędów")

7.3847886236e-39 6.5320031286e-39
5.5947082580e-02 6.1924818267e-01 
1.6700358609e-01 7.5799554823e-02 
2.4953561889e-02 2.0627139971e-02 
9.8213892433e-04 3.6484341870e-03 
3.0881196664e-04 4.7325653532e-05 
1.0222625675e-05 1.6300110410e-05 
5.2240228776e-07 7.5276149668e-07 
3.4008732060e-08 7.0807096352e-09 
2.0734535184e-10 1.0528978317e-09 
2.2828914583e-11 1.5384261453e-11 
4.9829894797e-13 3.2652718058e-13 
2.0052883271e-15 1.0838106499e-14 
1.7231438471e-16 3.7399653035e-17 
1.4112420216e-18 2.0188776573e-18 
1.6218281406e-20 2.5603400487e-20 
3.2624583093e-22 5.5590913986e-23 
7.2644043715e-25 3.1459813049e-24 
2.3002325831e-26 1.6508820438e-26 
1.9095201269e-28 1.1781625466e-28 
2.5317596902e-31 1.5947288351e-30 
1.0300905982e-32 2.4974442906e-33 
3.7697169485e-35 5.1372341720e-35 
1.8779261918e-37 3.0263209349e-37 
8.0239540979e-39 4.0651336891e-40 
8.0239540979e-39 4.0651336891e-40 
8.0239540979e-39 4.0651336891e-40 
8.0239540979e-39 4.0651336891e-40 
8.0239540979e-39 4.06

LoadError: [91msyntax: unexpected )[39m

Widać, że otrzymane wartości są bardzo bliskie dokładności maszynowej, a zbieżność funkcji jest mniej więcej liniowa (widać to z wykresu). Jednak problemy pojawiają się, gdy próbujemy obliczać sinusa z liczb o dużym module.

### Przykład 1.2

In [4]:
MAX_ITER = 100

result = setprecision(() -> sin(BigFloat(1) + BigFloat(2)im), 1024)
res = BigFloat(real(result)), BigFloat(imag(result))

sin_complex_1(Comp((20, 30)))

LoadError: [91mMethodError: no method matching sin_complex_1(::Tuple{BigFloat,BigFloat})[0m
Closest candidates are:
  sin_complex_1(::Tuple{BigFloat,BigFloat}, [91m::Any[39m) at In[2]:4[39m

### Przykład 1.3

In [5]:
MAX_ITER = 100

sin_test = BigFloat("-5.27928741830534994455823051946624768831858914683347441394183340021204190223502902340035002421106443911371532946971818949297297670370676157713326046029358490635177628675265498156826642248716282306643750181493583410429248706394483212324035268881811e12"), 
           BigFloat("8.24202099193480656045216406562863037108143065042992723430305830576936687747316708340911321957381143717872348723802699500386387652371839071850687423352690167260895903450486198737568149967049169497517663231718768889424321797283362454391848983979e11")

sin_complex_1(Comp((30, 30)))

LoadError: [91mMethodError: no method matching sin_complex_1(::Tuple{BigFloat,BigFloat})[0m
Closest candidates are:
  sin_complex_1(::Tuple{BigFloat,BigFloat}, [91m::Any[39m) at In[2]:4[39m

In [6]:
function sin_complex_1(z)
    x = z[1]
    y = z[2]
    as = a = Prec(1)
    bs = b = z[1]
    cs = c = Prec(1)
    ds = d = z[2]
    for i = 1:MAX_SIN_ITER
        a = - a * z[1] / prec(2  * i - 1) * z[1] / prec(2 * i)
        b = - b * z[1] / prec(2 * i) * z[1] / prec(2 * i + 1)
        c = c * z[2] / prec(2  * i - 1) * z[2] / prec(2 * i)
        d = d * z[2] / prec(2 * i) * z[2] / prec(2 * i + 1)
        as += a
        bs += b
        cs += c
        ds += d
        @printf "%.8f, %.8f\n" cs * bs as * ds
    end
    return cs * bs, as * ds
end

@show sin_1(12, -6, 30, Float32)
print("\n")
@show sin_1(0.3, -6, 30, Float32)

LoadError: [91mUndefVarError: sin_1 not defined[39m

In [7]:
function square(x)
    return x * x
end

# Funkcja obliczająca szybko kolejne potęgi. Zachowuje precyzję x (typ x = typ wartości zwracanej)
function power(x, n)
    if n > 0
        if n & 1 == 1
            return square(power(x, n>>=1)) * x
        else 
            return square(power(x, n>>=1))
        end
    else 
        return 1
    end
end

# Funkcja obliczająca sinusa z liczby rzeczywistej x korzystając
# z rozwinięcia w szereg Taylora długości n
function sin_real(x::Prec)
    δx = x
    Σ = x
    for i = 1:SIN_MAX_ITER
        δx = - δx * x / (2 * i) * x / (2 * i + 1)
        Σ += δx
    end
    return Σ
end

# Funkcja obliczająca cosinusa z liczby rzeczywistej x korzystając
# z rozwinięcia w szereg Taylora długości n
function cos_real(x::Prec)
    δx = Prec(1)
    Σ = Prec(1)
    for i = 1:COS_MAX_ITER
        δx = Prec(- δx  * x / (2i - 1) * x / (2i))
        Σ += δx
    end
    return Σ
end

function sin_nx(nx, n)
    Σ = Prec(0)
    x = Prec(nx) / prec(n)
    cos = cos_real(x)
    sin = sin_real(x)
    for i = 0:div(n, 2)
        Σ += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * power(cos, n - 2 * i - 1) * power(sin, 2 * i + 1)
    end
    return Σ
end

function cos_nx(nx, n)
    Σ = Prec(0)
    x = Prec(nx) / Prec(n)
    cos = cos_real(x)
    sin = sin_real(x)
    for i = 0:div(n, 2)
        Σ += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i) * power(cos, n - 2 * i) * power(sin, 2 * i)
    end
    return Σ
end


cos_nx (generic function with 1 method)

# Sposób 3

### Obliczanie sinusa metodą rekurencyjnych wzorów bez użycia pochodnych

In [8]:
setprecision(BigFloat, 128)

function ilosc_cyfr(x)
    ilosc = 0
    while x > 0
        ilosc = ilosc + 1
        x = x / 10
    end
    
    return ilosc
end

function msqrt(x, iter)
    start = x / 2 # (x / (10^ilosc_cyfr(x))) + 1 # Mozna ulepszyc wykorzystujac 10^ilosc_cyfr(x)
    
    x_n = BigFloat(start)
    for i = 0:iter
       x_n = 1/2 * (x_n + x/x_n)
    end
    
    return x_n
end

function sincos(iter)
    cn = -1 # cos pi
    sn = 0 # sin pi
    
    for i = 0:iter
        cn = msqrt((1 + cn)/2, 1000)
        sn = msqrt((1 - sn)/2, 1000)
    end
    
    @show cn
    @show sn
end

function rozwiniecie_binarne(x)
    ulam = BigFloat(x)
    t = Int32[]
    
    for i = 1:1000
        if ulam == 0
            @printf "1"
            break
        end
            
        ulam = BigFloat(BigFloat(ulam) * BigFloat(2))
        if (ulam >= 1)
            ulam = BigFloat(ulam - 1)
            push!(t, 1)
            @printf "%lf \t 1 \t %d\n" ulam i
        else
            push!(t, 0)
            @printf "%lf \t 0 \t %d\n" ulam i
        end
    end
    
    return t
end

# t jest rozwinieciem binarnym α/π
function ograniczenie(t, maxiter)
    Σ = BigFloat(0)
    l = length(t)
    
    cn = -1
    sn = 0
    for i = 1:l
        cn = msqrt((1 + cn)/2, 100)
        sn = msqrt((1 - cn)/2, 100)
        # @show cn sn
        
        @show Σ
        Σ = Σ + t[i] * BigFloat(π)/(2^i)
    end
    
    return BigFloat(sin(Σ)), BigFloat(sin(Σ + π/(2^l)))
end

# sinus bez pochodnych z wzorami rekurencyjnymi
function sinwd(α)
    x = BigFloat(BigFloat(α)/BigFloat(π))
    t = rozwiniecie_binarne(x)
    return ograniczenie(t, 500)
end

# Szybka aproksymacja sinusa, która ma na celu wydajność
# Kąt alpha musi być znormalizowany w postaci od [-π, π]
function rpdsin(angle)
    # normalizujemy kąt do przedziału [-π, π]
    if (angle < -π)
        angle += 2*π
    elseif (angle > π)
        angle -= 2*π
    end
        
    if (angle < 0)
        sin = 1.27323954 * angle + .405284735 * angle * angle;

        if (sin < 0)
            sin = .225 * (sin *-sin - sin) + sin;
        else
            sin = .225 * (sin * sin - sin) + sin;
        end
    else
        sin = 1.27323954 * angle - 0.405284735 * angle * angle;

        if (sin < 0)
            sin = .225 * (sin *-sin - sin) + sin;
        else
            sin = .225 * (sin * sin - sin) + sin;
        end
    end
end
    
@show msqrt(5, 1000)
@show sincos(100)

@show "Porównanie szybkiego liczenia z wzorami rekurencyjnymi"
@show sinwd(0.49)
@show rpdsin(1)

msqrt(5, 1000) = 2.23606797749978969640917366873127623544
cn = BigFloat(NaN, 128)
sn = 5.000000000000000000000000000001788544277e-01
sincos(100) = 5.000000000000000000000000000001788544277e-01
"Porównanie szybkiego liczenia z wzorami rekurencyjnymi" = "Porównanie szybkiego liczenia z wzorami rekurencyjnymi"
0.311944 	 0 	 1
0.623887 	 0 	 2
0.247775 	 1 	 3
0.495550 	 0 	 4
0.991099 	 0 	 5
0.982198 	 1 	 6
0.964396 	 1 	 7
0.928792 	 1 	 8
0.857584 	 1 	 9
0.715168 	 1 	 10
0.430337 	 1 	 11
0.860674 	 0 	 12
0.721348 	 1 	 13
0.442696 	 1 	 14
0.885392 	 0 	 15
0.770783 	 1 	 16
0.541567 	 1 	 17
0.083134 	 1 	 18
0.166268 	 0 	 19
0.332535 	 0 	 20
0.665071 	 0 	 21
0.330142 	 1 	 22
0.660283 	 0 	 23
0.320566 	 1 	 24
0.641132 	 0 	 25
0.282264 	 1 	 26
0.564528 	 0 	 27
0.129056 	 1 	 28
0.258113 	 0 	 29
0.516226 	 0 	 30
0.032451 	 1 	 31
0.064903 	 0 	 32
0.129806 	 0 	 33
0.259612 	 0 	 34
0.519224 	 0 	 35
0.038447 	 1 	 36
0.076894 	 0 	 37
0.153789 	 0 	 38
0.307577 	 0 	 3

0.8421677211675824

<h2>Test #1</h2>
Sprawdzamy, dla jakiego n skracanie przedziału \[0, &pi;/2) korzystając ze wzoru na sinus zwielokrotnionego kąta ma sens.

In [9]:
tests = [Float32(0.321563716), Float32(0.034321248), Float32(1.0523452), Float32(0.00000123456), Float32(1.000003123)]
range = 10000
max_n = 30
Prec = BigFloat

using PyPlot
xPlot = BigFloat[]
yPlot = BigFloat[]

for k = 1:max_n
    Σ² = 0
    for i = 1:5
    curr_test = tests[i]
        for j = 1:range
            curr_test = nextfloat(curr_test)
            target = sin(Float64(curr_test))
            Δ = target - sin_nx(curr_test, k, 10)
            Σ² += Δ * Δ
        end
    end
    @printf "n = %d, σ = %e \n" k sqrt(Σ² / range)
    push!(xPlot, k)
    push!(yPlot, sqrt(Σ² / range))
end

plot(xPlot, yPlot)

LoadError: [91mArgumentError: Module PyPlot not found in current path.
Run `Pkg.add("PyPlot")` to install the PyPlot package.[39m

In [10]:
k = 100
n = k / 2
x = Float64(1.0)
while x > 0.5
    x *= k * k / (2n - 1) / (2n)
    n += 1
end

@show n

n = 57.0


57.0

In [11]:
exact_sinh = Prec("10.0178749274099018989745936194658280601781041231828634644056532510463926051808870905252214580081921788136031436005276604654731845463086661965545666809253301856979834214884134800415226711700580868934289537240194339117071445876738421988703293310666662074873025393922419074202969904427384978934547999669783367996478777737387515158099245608102468799707910743158654767792319545286680090845441765466693103005564580755152769830821020898523711510472417274773015585649780529003697102125038131242050195636504635373129836335784769819562604683637035572605572218826029004693336598037776759038456992068734019962315072590278266626183660525078196734326386033216758620007131926640350061262077283570845419590562070693093134140875714806032858928269295308563453677379062041720184214492393331096756140382287179704817388080282537490855424255503455845715185625683972134857957278057234162657813792323741991")

function sinh_real(x::Prec)
    Σ = x
    δx = x
    for i = 1:SINH_MAX_ITER
        δx = δx * x / (2 * i) * x / (2 * i + 1)
        Σ += δx
        @show Σ - exact_sinh
    end
    return Σ
end

function cosh_real(x::Prec)
    Σ = Prec(1)
    δx = Prec(1)
    for i = 1:COSH_MAX_ITER
        δx = δx * x / (2 * i - 1) * x / (2 * i)
        Σ += δx
        @show Σ
    end
    return Σ
end

function sin_complex_n(z, n)
    Σ_r = Prec(0)
    Σ_i = Prec(0)
    cos = cos_complex(z)
    sin = sin_complex(z)
    for i = 0:div(n, 2)
        σ = multiply_complex(power_complex(cos, n - 2 * i - 1), power_complex(sin, 2 * i + 1))
        Σ_r += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * σ[1]
        Σ_i += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * σ[2]
    end
    return Σ_r, Σ_i
end


function sin_complex(z)
    return sin_real(z[1]) * cosh_real(z[2]), cos_real(z[1]) * sinh_real(z[2])
end

z = Prec(1), Prec(3)
exact_result = Prec("8.471645454300149424897583639976881207279850599273339864612249252167060156811179"), 
            Prec("5.41268092317819278427941063379302764217781251017215863262723484500606267062513")
z = sin_complex(z)
@show z[1] - exact_result[1]
@show z[2] - exact_result[2]

@show exact_sinh - nextfloat(exact_sinh)

Σ = 5.500000000000000000000000000000000000000
Σ = 8.875000000000000000000000000000000000000
Σ = 9.887499999999999999999999999999999999991
Σ = 1.005022321428571428571428571428571428569e+01
Σ = 1.006649553571428571428571428571428571427e+01
Σ = 1.006760501217532467532467532467532467532e+01
Σ = 1.006765987639592550306836021121735407448e+01
Σ = 1.006766193380419803410874839446268017695e+01
Σ = 1.006766199431620604972758334102871917999e+01
Σ = 1.006766199574938518693960837923686220901e+01
Σ = 1.006766199577730426104114133452663122904e+01
Σ = 1.006766199577775946333627502401505137609e+01
Σ = 1.006766199577776576613728456740796796275e+01
Σ = 1.006766199577776584117062991911502649355e+01
Σ = 1.006766199577776584194683693999475468525e+01
Σ = 1.006766199577776584195387914078902641279e+01
Σ = 1.006766199577776584195393562903069169402e+01
Σ = 1.00676619957777658419539360325181321603e+01
Σ = 1.006766199577776584195393603510091093993e+01
Σ = 1.006766199577776584195393603511581158673e+01
Σ = 1.0067661

-4.701977403289150031874946148888982711275e-38